# Log-In to the SDC
This notebook will walk through learning how to log-in to the [MMS Science Data Center](https://lasp.colorado.edu/mms/sdc/public/) using components of its [API](https://lasp.colorado.edu/mms/sdc/public/about/how-to/).

**Outline**
* [Obtain Cookie Credentials with Manual Redirects](#manual_redirects)
  * 9-Step Process
  * Session cookie saved after successful log-in
* [Obtain Cookie Credentials with Automated Redirects](#auto_redirects)
  * 4 Attempts
* [Download Data](#download_data)
* [Latis Log-In](#latis_login)
* [Conclusions](#conclusions)

<a id='manual_redirects'></a>
## Obtain Cookie Credentials with Manual Redirects
Here, we will try to download the MMS trigger data numbers (TDNs), which are an L1A data product. Access is restricteed to MMS team members and, hence, log-in is required. We first start by setting up the MMS_SDC_API class to attemp a download of FIELDS TDNs.

In [2]:
import requests
import urllib3

# Disable warnings (we are not using certificates)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Log-In Credentials
username = ''
password = ''
if (username == '') | (password == ''):
    raise ValueError('Must provide username and password.')

### Initial Attempt
We start by attempting to access the team-side of the SDC. This redirects us to a log-in page. To figure out what is going on, we will set `verify=False` because we do not have a verifcation certificate, and set `allow_redirects=False` in order to catch the redirect page.

In [3]:
# Function to examine responses
def show_response(r):
    '''Print attributes of the response function.'''
    print('Status: ', r.status_code)
    print('Cookies: ', r.cookies.get_dict())
    print('Headers:')
    for key in r.headers:
        print('\t', key+':', r.headers[key])

In [4]:
# Attempt to access the site
url0 = 'https://lasp.colorado.edu/mms/sdc/team/'
r = requests.get(url0, verify=0, allow_redirects=False)
print('URL: ', url0)
show_response(r)

# Store the re-direct information
url1 = r.headers['Location']

URL:  https://lasp.colorado.edu/mms/sdc/team/
Status:  302
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:29:18 GMT
	 Expires: Wed, 01 Jan 1997 12:00:00 GMT
	 Cache-Control: private,no-store,no-cache,max-age=0
	 Location: https://lasp-login.colorado.edu/idp/profile/SAML2/Redirect/SSO?SAMLRequest=fZJdb4IwFIb%2FCuk91CIz2IiJ04uZuGmE7WI3S4GDNikt6yn7%2BvVD0UWzxOu%2Bfd5znpwJilo1fNa6vd7CewvovK9aaeTHh4S0VnMjUCLXogbkruDp7HHFw2DAG2ucKYwi3gwRrJNGz43Gtgabgv2QBTxvVwnZO9cgp1QJbIIubqwoTQBlS9O9zHOjwO0DREMP4JBu1mlGvEU3idTiwLwm%2BMrspL7myLKh3TCVVHCCbKGUFgpH03RNvOUiIW9VdDfO41EZD8KcFTkTUVUNhyOAOGKCVVUXQ2xhqdEJ7RISDljsDyKfjTMW8nDMWfxKvM1p53upS6l3twXlfQj5Q5Zt%2FH6xF7B4XKoLkOnkoJkfi%2B2F%2BNtYcbZNpmczZf3jY2czDP5bxj%2FLE3pR13c3%2FKnjLxcbo2Tx7c2UMp9zC8JBQhih0%2F7L9YVMfwE%3D&RelayState=ss%3Amem%3A4c2df31a922523d530f29727cee4845ab9fc9def5435d6a5e11269ae2807c598
	 Content-Length: 897
	 Content-Type: text/html; charset=iso-8859-1


### Redirect #1
Now we will post to the next redirect page. This time we are given cookies that we must hold onto.

In [5]:
# Attempt to access the next page
r = requests.get(url1, verify=False, allow_redirects=False)
print('URL: ', url1)
show_response(r)

# Save cookies and redirect
url2 = r.headers['Location']
cookies1 = r.cookies


URL:  https://lasp-login.colorado.edu/idp/profile/SAML2/Redirect/SSO?SAMLRequest=fZJdb4IwFIb%2FCuk91CIz2IiJ04uZuGmE7WI3S4GDNikt6yn7%2BvVD0UWzxOu%2Bfd5znpwJilo1fNa6vd7CewvovK9aaeTHh4S0VnMjUCLXogbkruDp7HHFw2DAG2ucKYwi3gwRrJNGz43Gtgabgv2QBTxvVwnZO9cgp1QJbIIubqwoTQBlS9O9zHOjwO0DREMP4JBu1mlGvEU3idTiwLwm%2BMrspL7myLKh3TCVVHCCbKGUFgpH03RNvOUiIW9VdDfO41EZD8KcFTkTUVUNhyOAOGKCVVUXQ2xhqdEJ7RISDljsDyKfjTMW8nDMWfxKvM1p53upS6l3twXlfQj5Q5Zt%2FH6xF7B4XKoLkOnkoJkfi%2B2F%2BNtYcbZNpmczZf3jY2czDP5bxj%2FLE3pR13c3%2FKnjLxcbo2Tx7c2UMp9zC8JBQhih0%2F7L9YVMfwE%3D&RelayState=ss%3Amem%3A4c2df31a922523d530f29727cee4845ab9fc9def5435d6a5e11269ae2807c598
Status:  302
Cookies:  {'JSESSIONID': 'BE7DC41D0FC79191D9BDEEAE31C5FA38', '_idp_authn_lc_key': '7936e8b1-de7e-41f8-ac83-39e70db6f476'}
Headers:
	 Date: Thu, 19 Apr 2018 12:29:19 GMT
	 Set-Cookie: JSESSIONID=BE7DC41D0FC79191D9BDEEAE31C5FA38; Path=/idp; Secure, _idp_authn_lc_key=7936e8b1-de7e-41f8-ac83-39e70db6f476; Version=1; Path=/idp; Secure
	 Expires: 0
	 Cache-Con

### Redirect #2
Follow the redirected, poviding the cookies from the last page. Without the cookies, we are redirected to an error page saying that cookies are required to access the site.

In [6]:
# Attempt to access the site
r = requests.get(url2, verify=False, allow_redirects=False, cookies=cookies1)
print('URL: ', url2)
show_response(r)

# Store redirect information
url3 = r.headers['Location']

URL:  https://lasp-login.colorado.edu:443/idp/AuthnEngine
Status:  302
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:29:19 GMT
	 Expires: 0
	 Cache-Control: no-cache, no-store, must-revalidate, max-age=0
	 Pragma: no-cache
	 Location: https://lasp-login.colorado.edu:443/idp/Authn/UserPassword
	 Content-Length: 0
	 Connection: close
	 Content-Type: text/plain; charset=UTF-8


### Redirect #3
Again, follow the redirect.

In [7]:
# Access the page
r = requests.get(url3, verify=False, allow_redirects=False, cookies=cookies1)
print('URL: ', url3)
show_response(r)
print('Text:', r.text)

URL:  https://lasp-login.colorado.edu:443/idp/Authn/UserPassword
Status:  200
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:29:20 GMT
	 Expires: 0
	 Cache-Control: no-cache, no-store, must-revalidate, max-age=0
	 Pragma: no-cache
	 Content-Type: text/html; charset=UTF-8
	 Content-Length: 1820
	 Connection: close
Text: 
<html>
  <head>
    <title>LASP Login</title>
    <link rel="stylesheet" type="text/css" href="/idp/login.css"/>
  </head>

<body>
  <div id="container">
    <a href="http://lasp.colorado.edu/"><img src="/idp/images/logo.jpg" alt="LASP"/></a>
    <div class="loginbox">
       <h1>Sign In</h1>
       <div class="leftpane">
           
           
             <form action="/idp/Authn/UserPassword" method="post">
           
           <table>
             <tr><td width="40%"><label for="username">Username:</label></td><td><input name="j_username" type="text" id="username" autocapitalize="off" /></td></tr>
             <tr><td><label for="password">Password:</label></t

### Send Log-In Credentials
This time we get a status code of 200, meaning everything is ok. Reading through the text of the webpage, it looks like our username and password have to be submitted in association with the keys `'j_username'` and `'j_password'`. Let's try to post them. 

In [8]:
# Send login credentials
payload = {'j_username': username, 'j_password': password}
r = requests.post(url3, verify=False, allow_redirects=False, cookies=cookies1, data=payload)
print('URL: ', url3)
show_response(r)

# Update cookies
cookies2 = r.cookies
all_cookies = cookies1
all_cookies.update(cookies2)

# Next redirect
url4 = r.headers['Location']

URL:  https://lasp-login.colorado.edu:443/idp/Authn/UserPassword
Status:  302
Cookies:  {'_idp_session': 'MTMyLjE3Ny4yNDQuNjY%3D%7CYzYyMGI1Y2RkMTRkNGVjNTcxOGRkYWIwY2QzN2Y5NTQzMDg3MmZkMTg5Nzg0ZjUxMWYxYWMxODNkZWU2ZjMzYQ%3D%3D%7CIUOLJDKJ0iXwgLQKfP57JkaxwsM%3D'}
Headers:
	 Date: Thu, 19 Apr 2018 12:29:20 GMT
	 Expires: 0
	 Cache-Control: no-cache, no-store, must-revalidate, max-age=0
	 Pragma: no-cache
	 Set-Cookie: _idp_session=MTMyLjE3Ny4yNDQuNjY%3D%7CYzYyMGI1Y2RkMTRkNGVjNTcxOGRkYWIwY2QzN2Y5NTQzMDg3MmZkMTg5Nzg0ZjUxMWYxYWMxODNkZWU2ZjMzYQ%3D%3D%7CIUOLJDKJ0iXwgLQKfP57JkaxwsM%3D; Version=1; Path=/idp; Secure
	 Location: https://lasp-login.colorado.edu:443/idp/profile/SAML2/Redirect/SSO
	 Content-Length: 0
	 Connection: close
	 Content-Type: text/plain; charset=UTF-8


### Verify Log-In Status

In [9]:
# Check page again, now with log-in cookies
r = requests.get(url4, verify=False, allow_redirects=False, cookies=all_cookies)
print('URL: ', url4)
show_response(r)
print('Text:', r.text)

URL:  https://lasp-login.colorado.edu:443/idp/profile/SAML2/Redirect/SSO
Status:  200
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:29:21 GMT
	 Expires: 0
	 Cache-Control: no-cache, no-store
	 Pragma: no-cache
	 Set-Cookie: _idp_authn_lc_key=7936e8b1-de7e-41f8-ac83-39e70db6f476; Version=1; Max-Age=0; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/idp
	 Content-Type: text/html;charset=UTF-8
	 Connection: close
	 Transfer-Encoding: chunked
Text: 
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en">

    <body onload="document.forms[0].submit()">
        <noscript>
            <p>
                <strong>Note:</strong> Since your browser does not support JavaScript,
                you must press the Continue button once to proceed.
            </p>
        </noscript>
        
        <form action="https&#x3a;&#x2f;&#x2f;lasp.colorado.edu&#x2f;Shibboleth.sso&#x2f;SAML2&#x2f;POST" method="post">
            <div>
                <input type="hidden" name="RelayState" value="ss&#x3a

### Parse Form
Now we have to read the form and click the "Continue" button. To do so find the form addres, then search for the input key-value pairs.

In [10]:
# Function to parse the form containing the "Continue" button
def parse_form(r):
    '''Parse key-value pairs from the log-in form
    
    Parameters
    ----------
    r (object):    requests.response object.
    
    Returns
    -------
    form (dict):   key-value pairs parsed from the form.
    '''
    # Find action URL
    pstart = r.text.find('<form')
    pend = r.text.find('>', pstart)
    paction = r.text.find('action', pstart, pend)
    pquote1 = r.text.find('"', pstart, pend)
    pquote2 = r.text.find('"', pquote1+1, pend)
    url5 = r.text[pquote1+1:pquote2]
    url5 = url5.replace('&#x3a;', ':')
    url5 = url5.replace('&#x2f;', '/')
    print('URL:', url5)

    # Parse values from the form
    pinput = r.text.find('<input', pend+1)
    inputs = {}
    while pinput != -1:
        # Parse the name-value pair
        pend = r.text.find('/>', pinput)

        # Name
        pname = r.text.find('name', pinput, pend)
        pquote1 = r.text.find('"', pname, pend)
        pquote2 = r.text.find('"', pquote1+1, pend)
        name = r.text[pquote1+1:pquote2]

        # Value
        if pname != -1:
            pvalue = r.text.find('value', pquote2+1, pend)
            pquote1 = r.text.find('"', pvalue, pend)
            pquote2 = r.text.find('"', pquote1+1, pend)
            value = r.text[pquote1+1:pquote2]
            value = value.replace('&#x3a;', ':')

            # Extract the values
            inputs[name] = value

        # Next iteraction
        pinput = r.text.find('<input', pend+1)
    
    form = {'url': url5,
            'payload': inputs}
    
    return form

In [12]:
# Read the form
form = parse_form(r)
print('Form: ', form) 

URL: https://lasp.colorado.edu/Shibboleth.sso/SAML2/POST
Form:  {'url': 'https://lasp.colorado.edu/Shibboleth.sso/SAML2/POST', 'payload': {'RelayState': 'ss:mem:4c2df31a922523d530f29727cee4845ab9fc9def5435d6a5e11269ae2807c598', 'SAMLResponse': 'PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz48c2FtbDJwOlJlc3BvbnNlIHhtbG5zOnNhbWwycD0idXJuOm9hc2lzOm5hbWVzOnRjOlNBTUw6Mi4wOnByb3RvY29sIiBEZXN0aW5hdGlvbj0iaHR0cHM6Ly9sYXNwLmNvbG9yYWRvLmVkdS9TaGliYm9sZXRoLnNzby9TQU1MMi9QT1NUIiBJRD0iXzU4YmE3ZDc5NzdmNDRiYTJmNTAwMzEzZWJiMTdlNzA0IiBJblJlc3BvbnNlVG89Il9mNDU5Yjg2ZDgwMmIxY2IxYTRmZjMzNmVlODQxYTFmZiIgSXNzdWVJbnN0YW50PSIyMDE4LTA0LTE5VDEyOjI5OjIxLjA1MFoiIFZlcnNpb249IjIuMCI+PHNhbWwyOklzc3VlciB4bWxuczpzYW1sMj0idXJuOm9hc2lzOm5hbWVzOnRjOlNBTUw6Mi4wOmFzc2VydGlvbiIgRm9ybWF0PSJ1cm46b2FzaXM6bmFtZXM6dGM6U0FNTDoyLjA6bmFtZWlkLWZvcm1hdDplbnRpdHkiPmh0dHBzOi8vbGFzcC1sb2dpbi5jb2xvcmFkby5lZHUvaWRwL3NoaWJib2xldGg8L3NhbWwyOklzc3Vlcj48c2FtbDJwOlN0YXR1cz48c2FtbDJwOlN0YXR1c0NvZGUgVmFsdWU9InVybjpvYXNpczpuYW1lczp0YzpTQU1MOjI

### Click Continue
Submitting the form to continue will return the session cookies. These can be submitted with future posts to the team site without having to log back in again.

In [15]:
# Submit the form
r = requests.post(form['url'], verify=False, allow_redirects=False, data=form['payload'])
print('URL: ', form['url'])
show_response(r)
print('Text:', r.text)

URL:  https://lasp.colorado.edu/Shibboleth.sso/SAML2/POST
Status:  302
Cookies:  {'_shibsession_64656661756c7468747470733a2f2f646d7a2d73686962322e6c6173702e636f6c6f7261646f2e6564752f73686962626f6c657468': '_ff7fd018452a2659d168d1c8d626c111'}
Headers:
	 Date: Thu, 19 Apr 2018 12:31:17 GMT
	 Server: Apache/2.2.15 (Red Hat)
	 Set-Cookie: _shibsession_64656661756c7468747470733a2f2f646d7a2d73686962322e6c6173702e636f6c6f7261646f2e6564752f73686962626f6c657468=_ff7fd018452a2659d168d1c8d626c111; path=/; secure; HttpOnly
	 Expires: Wed, 01 Jan 1997 12:00:00 GMT
	 Cache-Control: private,no-store,no-cache,max-age=0
	 Location: https://lasp.colorado.edu/mms/sdc/team/
	 Content-Length: 308
	 Content-Type: text/html; charset=iso-8859-1
Text: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>302 Found</title>
</head><body>
<h1>Found</h1>
<p>The document has moved <a href="https://lasp.colorado.edu/mms/sdc/team/">here</a>.</p>
<hr>
<address>Apache/2.2.15 (Red Hat) Server at lasp.co

### Save Cookies for Later
Since the cookie dictionary has weird keys, save as a key-value pair as a tuple.

In [16]:
key = r.cookies.keys()[0]
value = r.cookies[key]
session_cookies = (key, value)
print(session_cookies)

('_shibsession_64656661756c7468747470733a2f2f646d7a2d73686962322e6c6173702e636f6c6f7261646f2e6564752f73686962626f6c657468', '_ff7fd018452a2659d168d1c8d626c111')


<a id='auto_redirects'></a>
## Obtain Cookie Credentials with Automated Redirects
This time, we will do the same thing, but set `allow_redirects=True` to see if we can avoid a few steps.

### Initial Request

In [17]:
# Attempt to access the site
#   - Each of the redirects are stored in the history attribute
url0 = 'https://lasp.colorado.edu/mms/sdc/team/'
r = requests.get(url0, verify=0)
print('URL: ', url0)
show_response(r)
for idx, response in enumerate(r.history):
    print('\nRESPONSE #', idx+1, ':')
    show_response(response)

URL:  https://lasp.colorado.edu/mms/sdc/team/
Status:  200
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:31:53 GMT
	 Expires: 0
	 Cache-Control: no-cache, no-store, must-revalidate, max-age=0
	 Pragma: no-cache
	 Content-Type: text/html; charset=UTF-8
	 Content-Length: 1820
	 Connection: close

RESPONSE # 1 :
Status:  302
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:31:52 GMT
	 Expires: Wed, 01 Jan 1997 12:00:00 GMT
	 Cache-Control: private,no-store,no-cache,max-age=0
	 Location: https://lasp-login.colorado.edu/idp/profile/SAML2/Redirect/SSO?SAMLRequest=fZJdb4IwFIb%2FCuk9VPBj0giJ04uZuEmE7WI3S4GDNikt6yn7%2BvVD0UWzxOu%2Bfd5znpwZ8lo2bN7avdrCewtona9aKmTHh4i0RjHNUSBTvAZktmDp%2FHHNAm%2FAGqOtLrQkzhwRjBVaLbTCtgaTgvkQBTxv1xHZW9sgo1RybLwurg0vtQdlS9O9yHMtwe49RE0P4IAmmzQjzrKbRCh%2BYF4TXKl3Ql1zRNnQbphKSDhBtlAKA4WlabohzmoZkbcpr8KygLwa5ONJ5ed3k8k4HE7DaeHzUVD5XQyxhZVCy5WNSDDwp%2B5g5Pph5gds6LNx8Eqc5LTzvVClULvbgvI%2BhOwhyxK3X%2BwFDB6X6gIknh00s2OxuRB%2FG8vPtkl8NlPWPy52NgPvv2X8szyjF3V9d8OeOv5qmWg

### Submit Username & Password
With the redirect history, we can capture the cookies from the second redirect, then post our log-in information to the URL from the final redirect.

In [18]:
# Information from history
cookies = r.history[1].cookies
url1 = r.history[2].headers['Location']

# Send login credentials
payload = {'j_username': username, 'j_password': password}
r = requests.post(url1, verify=False, allow_redirects=True, cookies=cookies, data=payload)
print('URL:         ', url1)
show_response(r)
print('Text:', r.text)

URL:          https://lasp-login.colorado.edu:443/idp/Authn/UserPassword
Status:  200
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:31:53 GMT
	 Expires: 0
	 Cache-Control: no-cache, no-store
	 Pragma: no-cache
	 Set-Cookie: _idp_authn_lc_key=457aeaed-d820-4a6d-b542-40c428ef234f; Version=1; Max-Age=0; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/idp
	 Content-Type: text/html;charset=UTF-8
	 Connection: close
	 Transfer-Encoding: chunked
Text: 
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en">

    <body onload="document.forms[0].submit()">
        <noscript>
            <p>
                <strong>Note:</strong> Since your browser does not support JavaScript,
                you must press the Continue button once to proceed.
            </p>
        </noscript>
        
        <form action="https&#x3a;&#x2f;&#x2f;lasp.colorado.edu&#x2f;Shibboleth.sso&#x2f;SAML2&#x2f;POST" method="post">
            <div>
                <input type="hidden" name="RelayState" value="ss&#x3a

### Parse Form & Click Continue
Now we have to parse the form in order to click "Continue"

In [19]:
# Parse the form
form = parse_form(r)

# Submit the form
r = requests.post(form['url'], verify=False, allow_redirects=True, cookies=cookies, data=form['payload'])
print('URL:         ', form['url'])
show_response(r)
for idx, response in enumerate(r.history):
    print('\nRESPONSE #', idx, ':')
    show_response(response)

URL: https://lasp.colorado.edu/Shibboleth.sso/SAML2/POST
URL:          https://lasp.colorado.edu/Shibboleth.sso/SAML2/POST
Status:  200
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:31:54 GMT
	 Server: Apache/2.2.15 (Red Hat)
	 Last-Modified: Tue, 03 Oct 2017 15:32:42 GMT
	 ETag: "64454-175f-55aa633b31a80"
	 Accept-Ranges: bytes
	 Content-Length: 5983
	 Cache-Control: no-cache, no-store, must-revalidate
	 Pragma: no-cache
	 Expires: 0
	 Content-Type: text/html; charset=UTF-8

RESPONSE # 0 :
Status:  302
Cookies:  {'_shibsession_64656661756c7468747470733a2f2f646d7a2d73686962322e6c6173702e636f6c6f7261646f2e6564752f73686962626f6c657468': '_a71ead0e9a3bd23411a114f34d6d3d79'}
Headers:
	 Date: Thu, 19 Apr 2018 12:31:54 GMT
	 Server: Apache/2.2.15 (Red Hat)
	 Set-Cookie: _shibsession_64656661756c7468747470733a2f2f646d7a2d73686962322e6c6173702e636f6c6f7261646f2e6564752f73686962626f6c657468=_a71ead0e9a3bd23411a114f34d6d3d79; path=/; secure; HttpOnly
	 Expires: Wed, 01 Jan 1997 12:00:00 GMT


### Extract Session Cookies
The last thing to do is save the session cookies for future `get`/`post` requests.

In [20]:
key = r.history[0].cookies.keys()[0]
value = r.history[0].cookies[key]
session_cookies = (key, value)
print(session_cookies)

('_shibsession_64656661756c7468747470733a2f2f646d7a2d73686962322e6c6173702e636f6c6f7261646f2e6564752f73686962626f6c657468', '_a71ead0e9a3bd23411a114f34d6d3d79')


<a id='download_data'></a>
## Download Data
Now we can use the session cookies to download data from the team site. Note that we have to use the [Browse the Data](https://lasp.colorado.edu/mms/sdc/team/about/browse/) file archive tree, not the [SDC API](https://lasp.colorado.edu/mms/sdc/team/about/how-to/).

In [28]:
# Write to temporary directory
import tempfile
import os

# Note: if data is reprocessed, this link will be dead.
host = 'https://lasp.colorado.edu/'
path = 'mms/sdc/team/about/browse/mms4/edi/srvy/l2/amb-alt-oob/2018/03/'
file = 'mms4_edi_srvy_l2_amb-alt-oob_20180326_v1.2.0.cdf'
url = host + path + file

# Download
print('Downloading...')
r = requests.get(url, cookies={session_cookies[0]: session_cookies[1]})
show_response(r)

# Write data to temporary file
fout = os.path.join(tempfile.gettempdir(), file)
print('Output File: ', fout)
with open(fout, 'wb') as f:
    f.write(r.content)

Status:  200
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 12:46:23 GMT
	 Server: Apache/2.2.15 (Red Hat)
	 Last-Modified: Thu, 19 Apr 2018 00:46:45 GMT
	 ETag: "aec72-16a2704-56a28e7e1a10b"
	 Accept-Ranges: bytes
	 Content-Length: 23734020
	 Access-Control-Allow-Origin: *
	 Content-Type: application/x-netcdf
Output File:  /var/folders/vx/c1d4m37n4h5gn_vtcz9hhck80000gn/T/mms4_edi_srvy_l2_amb-alt-oob_20180326_v1.2.0.cdf


<a id='latis_login'></a>
## Log-In Through Latis
As the following example shows, the session cookies obtained by logging in to the [team file browser](https://lasp.colorado.edu/mms/sdc/team/about/browse/) do not work with the [API](https://lasp.colorado.edu/mms/sdc/team/about/how-to/). After an unsuccessful attempt, I try to log-in through Latis, which the SITL does in the IDL SPEDAS distribution. However, I am unable to post my log-in credentials correctly.

In [31]:
# Create a query for the API
url = 'https://lasp.colorado.edu/mms/sdc/sitl/files/api/v1/download/science'
payload = {'sc_id': 'mms1',
           'instrument_id': 'edi',
           'data_rate_mode': 'srvy',
           'data_level': 'l2',
           'descriptor': 'amb-alt-oob',
           'start_date': '2018-03-26',
           'end_date': '2018-03-26'}

# Attempt to download the data with the previous session cookies
r = requests.get(url, params=payload, cookies={session_cookies[0]: session_cookies[1]})
print('URL: ', r.url)
show_response(r)

URL:  https://lasp.colorado.edu/mms/sdc/sitl/files/api/v1/download/science?sc_id=mms1&instrument_id=edi&data_rate_mode=srvy&data_level=l2&descriptor=amb-alt-oob&start_date=2018-03-26&end_date=2018-03-26
Status:  401
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 13:16:15 GMT
	 WWW-Authenticate: Basic realm="Secure Area"
	 Content-Length: 486
	 Content-Type: text/html; charset=iso-8859-1


In [32]:
# Latis site
url = 'https://lasp.colorado.edu/mms/sdc/sitl/latis/dap/properties.txt'

# Attempt to log-in
#   - I have also tried "j_username" and "j_password", as above
r = requests.post(url, data={'username': username, 'password': password})
show_response(r)

Status:  401
Cookies:  {}
Headers:
	 Date: Thu, 19 Apr 2018 13:20:19 GMT
	 WWW-Authenticate: Basic realm="Secure Area"
	 Content-Length: 486
	 Content-Type: text/html; charset=iso-8859-1


<a id='conclusions'></a>
# Conclusion
We first tried to log-in to the team [File Browser](https://lasp.colorado.edu/mms/sdc/team/about/browse/) and found that this sends us through a series of redirects before landing on the log-in page. Session cookies were successfully obtained by both manually and automatically following the redirects. Next, we tried to use the same session cookies to access data through the SDC's [API](https://lasp.colorado.edu/mms/sdc/team/about/how-to/) but were unsuccessful. After looking through the SITL software in the SPEDAS distribution, we found that the SITL posts log-in information to a Latis 